## 1. Data Import

In [16]:
import csv
import pandas as pd
import geopandas as gpd
import fiona
from os import listdir
import re

df = gpd.read_file('ej-0424/ej.shp')
#df = df.set_index('ID').fillna(0)
display(df.head())

,ID,ACSTOTPOP,ACSIPOVBAS,ACSEDUCBAS,ACSTOTHH,ACSTOTHU,MINORPOP,MINORPCT,LOWINCOME,LOWINCPCT,...,T_PM25_B6,T_PM25_P2,T_PM25_P6,AREALAND,AREAWATER,NPL_CNT,TSDF_CNT,Shape_Leng,Shape_Area,geometry
0,170310101001,639,639,515,312,376,443,0.693271,247,0.386541,...,29%ile,87%ile,79%ile,62463.0,0.0,0,0,1689.212502,113261.122684,POLYGON ((-9759460.328199999 5164183.183399998...
1,170310101002,1768,1751,1116,833,955,1202,0.679864,1217,0.695031,...,86%ile,95%ile,92%ile,183315.0,0.0,0,0,2659.049626,332392.122294,POLYGON ((-9760181.790100001 5164417.844300002...
2,170310101003,1981,1881,1529,1061,1292,994,0.501767,620,0.329612,...,69%ile,77%ile,67%ile,133973.0,0.0,0,0,3951.080271,242919.062756,"POLYGON ((-9759674.952299999 5164425.9362, -97..."
3,170310102011,1417,1417,1025,619,742,916,0.646436,704,0.496824,...,75%ile,89%ile,88%ile,95922.0,0.0,0,0,1786.028121,173888.451215,POLYGON ((-9760948.113600001 5163250.440399997...
4,170310102012,4641,4628,3064,1673,1810,3615,0.778927,2569,0.555099,...,98%ile,94%ile,93%ile,259326.0,0.0,0,0,3623.548693,470159.705671,"POLYGON ((-9761011.5658 5163899.829499997, -97..."


In [17]:
df.head()

,ID,ACSTOTPOP,ACSIPOVBAS,ACSEDUCBAS,ACSTOTHH,ACSTOTHU,MINORPOP,MINORPCT,LOWINCOME,LOWINCPCT,...,T_PM25_B6,T_PM25_P2,T_PM25_P6,AREALAND,AREAWATER,NPL_CNT,TSDF_CNT,Shape_Leng,Shape_Area,geometry
0,170310101001,639,639,515,312,376,443,0.693271,247,0.386541,...,29%ile,87%ile,79%ile,62463.0,0.0,0,0,1689.212502,113261.122684,POLYGON ((-9759460.328199999 5164183.183399998...
1,170310101002,1768,1751,1116,833,955,1202,0.679864,1217,0.695031,...,86%ile,95%ile,92%ile,183315.0,0.0,0,0,2659.049626,332392.122294,POLYGON ((-9760181.790100001 5164417.844300002...
2,170310101003,1981,1881,1529,1061,1292,994,0.501767,620,0.329612,...,69%ile,77%ile,67%ile,133973.0,0.0,0,0,3951.080271,242919.062756,"POLYGON ((-9759674.952299999 5164425.9362, -97..."
3,170310102011,1417,1417,1025,619,742,916,0.646436,704,0.496824,...,75%ile,89%ile,88%ile,95922.0,0.0,0,0,1786.028121,173888.451215,POLYGON ((-9760948.113600001 5163250.440399997...
4,170310102012,4641,4628,3064,1673,1810,3615,0.778927,2569,0.555099,...,98%ile,94%ile,93%ile,259326.0,0.0,0,0,3623.548693,470159.705671,"POLYGON ((-9761011.5658 5163899.829499997, -97..."


In [18]:
df.shape

(3992, 368)

In [19]:
# get field descriptions
df_dict = pd.read_excel('EJScreen_Index_DescriptionsV4_Pub.xlsx')

### 1*. remove the blockgroups in chicago

In [20]:
df_chicago = pd.read_csv('target_blockgroups.csv')

In [21]:
df_chicago.shape

(2179, 2)

In [22]:
df_chicago.head()

,id,Unnamed: 1
0,170310101001,NaN
1,170310101002,NaN
2,170310101003,NaN
3,170310102011,NaN
4,170310102012,NaN


In [23]:
df_chicago = df_chicago['id'].astype(str)

In [24]:
df_chicago.head()

0    170310101001
1    170310101002
2    170310101003
3    170310102011
4    170310102012
Name: id, dtype: object

In [25]:
df_chicago = df_chicago.tolist()

In [26]:
df = df.loc[~df["ID"].isin(df_chicago)]

In [27]:
df.shape

(1813, 368)

In [28]:
df = df.set_index('ID').fillna(0)

### 2. Process environmental columns

In [29]:
env_indicators = [
    'OZONE',
    'PM25']

env_dict = {
    'CANCER': 'CANCR',
    'PRE1960PCT': 'LDPNT'
}

#display(df_dict[df_dict.FIELD_NAME.isin(env_indicators)])

#### 2a. Calculate percentile

In [30]:
from scipy import stats

c_env_df = df.filter(env_indicators).apply(lambda series: series.apply(lambda x: stats.percentileofscore(series, x, kind='strict')))
c_env_df.rename(lambda x: 'C_' + env_dict.get(x,x), axis='columns', inplace=True)

display(c_env_df.head())

,C_OZONE,C_PM25
ID,,
170317702011,19.966906,35.466078
170317702012,19.966906,35.466078
170317702013,19.966906,35.466078
170317702021,24.986211,29.564258
170317702022,24.986211,29.564258


#### 2b. bin percentiles into quintiles

In [31]:
bins = [-1,20,40,60,80,100]
labels = [1,2,3,4,5]

s_env_df = c_env_df.apply(lambda series: pd.cut(series, bins, labels=labels).values.add_categories(0).astype(int))
s_env_df.rename(lambda x: x.replace('C_', 'S_'), axis='columns', inplace=True)

display(s_env_df.head())

,S_OZONE,S_PM25
ID,,
170317702011,1,2
170317702012,1,2
170317702013,1,2
170317702021,2,2
170317702022,2,2


#### 2c. sum up the qunitiles and bin them

In [32]:
s_env_df['S_ENV_TOTAL'] = s_env_df.sum(axis=1)

series = s_env_df['S_ENV_TOTAL']
s_env_df['C_ENV_TOTAL'] = series.apply(lambda x: stats.percentileofscore(series, x, kind='strict'))
s_env_df['S_ENV_OVERALL'] = pd.cut(s_env_df['C_ENV_TOTAL'], bins, labels=labels).astype(int)

display(s_env_df.head())

,S_OZONE,S_PM25,S_ENV_TOTAL,C_ENV_TOTAL,S_ENV_OVERALL
ID,,,,,
170317702011,1,2,3,0.000000,1
170317702012,1,2,3,0.000000,1
170317702013,1,2,3,0.000000,1
170317702021,2,2,4,4.412576,1
170317702022,2,2,4,4.412576,1


### 3. process demographic columns

In [33]:
pop_indicators = [
    'LOWINCPCT',
    'MINORPCT',
    'LESSHSPCT',
    'LINGISOPCT',
    'UNDER5PCT',
    'OVER64PCT'
]

pop_dict = {
    'LOWINCPCT': 'LWINCPCT',
    'MINORPCT': 'MINORPCT',
    'LESSHSPCT': 'LESHSPCT',
    'LINGISOPCT': 'LNGISPCT',
    'UNDER5PCT': 'UNDR5PCT',
    'OVER64PCT': 'OVR64PCT'
}

display(df_dict[df_dict.FIELD_NAME.isin(pop_indicators)])

,FIELD_NAME,DESCRIPTION,CATEGORY
7,MINORPCT,Pct. Minority Population,Population
9,LOWINCPCT,Pct. Low Income (<2x poverty level),Income/Poverty
11,LESSHSPCT,Pct. Less than High School Education,Education
13,LINGISOPCT,Pct. Linguistically Isolated,Language
15,UNDER5PCT,Pct. Under Age 5,Population
17,OVER64PCT,Pct. Over Age 64,Population


#### 3a. calculate percentiles

In [34]:

c_pop_df = df.filter(pop_indicators).apply(lambda series: series.apply(lambda x: stats.percentileofscore(series, x, kind='strict')))
c_pop_df.rename(lambda x: 'C_' + pop_dict.get(x,x), axis='columns', inplace=True)

display(c_pop_df.head())

,C_LWINCPCT,C_MINORPCT,C_LESHSPCT,C_LNGISPCT,C_UNDR5PCT,C_OVR64PCT
ID,,,,,,
170317702011,13.348042,43.739658,18.698290,69.387755,38.279095,73.469388
170317702012,30.226145,20.739106,34.859349,42.029785,33.425262,90.182019
170317702013,75.620518,45.835632,26.475455,77.992278,49.751793,47.435190
170317702021,54.715940,24.875896,21.290678,51.627137,71.263100,36.458908
170317702022,42.140099,8.825152,48.814120,64.920022,12.244898,93.381136


#### 3b. bin them into quintiles

In [35]:
bins = [-1,20,40,60,80,100]
labels = [1,2,3,4,5]

s_pop_df = c_pop_df.apply(lambda series: pd.cut(series, bins, labels=labels).values.add_categories(0).astype(int))
s_pop_df.rename(lambda x: x.replace('C_', 'S_'), axis='columns', inplace=True)

display(s_pop_df.head())

,S_LWINCPCT,S_MINORPCT,S_LESHSPCT,S_LNGISPCT,S_UNDR5PCT,S_OVR64PCT
ID,,,,,,
170317702011,1,3,1,4,2,4
170317702012,2,2,2,3,2,5
170317702013,4,3,2,4,3,3
170317702021,3,2,2,3,4,2
170317702022,3,1,3,4,1,5


#### 3c. sum up quintiles and bin them too

In [36]:
s_pop_df['S_POP_TOTAL'] = s_pop_df.sum(axis=1)

series = s_pop_df['S_POP_TOTAL']
s_pop_df['C_POP_TOTAL'] = series.apply(lambda x: stats.percentileofscore(series, x, kind='strict'))
s_pop_df['S_POP_OVERALL'] = pd.cut(s_pop_df['C_POP_TOTAL'], bins, labels=labels).astype(int)

display(s_pop_df.head())

,S_LWINCPCT,S_MINORPCT,S_LESHSPCT,S_LNGISPCT,S_UNDR5PCT,S_OVR64PCT,S_POP_TOTAL,C_POP_TOTAL,S_POP_OVERALL
ID,,,,,,,,,
170317702011,1,3,1,4,2,4,15,24.820739,2
170317702012,2,2,2,3,2,5,16,33.259790,2
170317702013,4,3,2,4,3,3,19,54.329840,3
170317702021,3,2,2,3,4,2,16,33.259790,2
170317702022,3,1,3,4,1,5,17,40.154440,3


### 4. merge everything

merged_df = pd.concat([df, c_pop_df, c_env_df, s_pop_df, s_env_df], axis=1, sort=True)
merged_df['S_OVERALL'] = merged_df['S_POP_OVERALL'] + merged_df['S_ENV_OVERALL']

display(merged_df.head())

### 5. export

merged_df.to_file('ej_suburban_v3/output_ej.shp')

In [37]:
merged_df = pd.concat([df, s_pop_df], axis=1, sort=True)
merged_df.to_file('0524 map/socio/suburban/suburban.shp')